#### Подключаемся к серверy

```bash
ssh 305_koryagin@37.139.32.56 -i ./id_rsa_305_koryagin.txt
```

#### Запускаем spark

```bash
/spark2.4/bin/pyspark
```

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("gogin_spark").getOrCreate()
```

## Создание DataFrame

```python
# для начала готовим DataFrame
data = spark.read.parquet("input_csv_for_recommend_system/data.parquet")
data.show(n=5, truncate=True)
```

<details>
    <summary> → вывод консоли SPARK</summary>
    
![Title](../images/2021-04-25_125511.jpg)

</details>